# Encoder-Decoder Translation

In [1]:
# !pip install transformers sentencepiece --quiet

In [2]:
from transformers import pipeline

In [3]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-fr")

C:\Users\ttaul\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

C:\Users\ttaul\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ttaul\.cache\huggingface\hub\models--Helsinki-NLP--opus-mt-en-fr. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

C:\Users\ttaul\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\marian\tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [4]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-fr")

In [6]:
text = "The bank of the river was covered with wildflowers."
translation = translator(text)
translation

[{'translation_text': 'La rive de la rivière était couverte de fleurs sauvages.'}]

In [10]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
model = AutoModelForSeq2SeqLM.from_pretrained(
    "Helsinki-NLP/opus-mt-en-fr",
    output_hidden_states=True
)

inputs = tokenizer(text, return_tensors="pt")

# Get encoder hidden states only
encoder_outputs = model.model.encoder(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    output_hidden_states=True,
    return_dict=True
)

In [11]:
for i, layer in enumerate(encoder_outputs.hidden_states):
    print(i, layer.shape)

0 torch.Size([1, 13, 512])
1 torch.Size([1, 13, 512])
2 torch.Size([1, 13, 512])
3 torch.Size([1, 13, 512])
4 torch.Size([1, 13, 512])
5 torch.Size([1, 13, 512])
6 torch.Size([1, 13, 512])


In [12]:
translated_tokens = model.generate(
    inputs["input_ids"],
    max_length=40,
    num_beams=4,
    early_stopping=True
)

print(tokenizer.decode(translated_tokens[0], skip_special_tokens=True))


La rive de la rivière était couverte de fleurs sauvages.
